In [ ]:
!pip install transformers
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 65.6 MB/s 
     |████████████████████████████████| 6.6 MB 47.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import nltk
import os
from random import shuffle
from random import randint
import re
import textdistance
nltk.download('punkt')

from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
from transformers import AutoConfig, AutoModelForMaskedLM


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelWithLMHead.from_pretrained("bert-large-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#reading the csv file containing the video id, the slur, the matching excerpts from the youtube and amazon transcripts, and the ground truth for the slur
df=pd.read_csv('CSV/Dataset_Amazon_Final_2.csv')

In [ ]:
df

,Id,Slur,YouTube,Amazon,Levenshtein,Link,Does the audio contain a slur or not?,Original Transcription,"Race of speaker (Caucasian, Asian, South Asian, Black, and Australoid/ Animation)",Actual Word
0,53,ass,ready for a rescue shes going to take to the s...,ready for a rescue shes going to take to the s...,0,https://www.youtube.com/watch?v=0Z9rIS4xNZ0&t=...,No,ready for a rescue shes going to take to the s...,Caucasian Woman,maya
1,218,bitch,his life hey bitch mean,his life hey bitch mean,0,https://www.youtube.com/watch?v=9R4lLcm5p7I&t=...,NaN,NaN,NaN,NaN
2,312,cocktail,also exceptionally strong for their size they ...,also exceptionally strong for their size they ...,0,https://www.youtube.com/watch?v=Fw1_1_yKZ78&t=...,No,also exceptionally strong for their size they ...,NaN,copter
3,326,crap,the red crap is standing on,the red crap is standing on,0,https://www.youtube.com/watch?v=pmQTB3ScUo8&t=...,No,The red crab is standing on,NaN,crab
4,360,crap,stretchy and sticky and now we have a crap and...,stretchy and sticky and now we have a crap and...,0,https://www.youtube.com/watch?v=jrZwy-ybyfA&t=...,No,stretchy and sticky and now we have a crab and...,NaN,crab
...,...,...,...,...,...,...,...,...,...,...
1654,2110,shit,just for fun climb and ten nine ten dont you w...,just for fun fine and 10 9 and 10 dont you wan...,2643,https://www.youtube.com/watch?v=zwibXy4CCgo&t=...,NaN,NaN,NaN,NaN
1655,1862,rape,spread on the door five and six take your pick...,spread on the door five and six take your pick...,3246,https://www.youtube.com/watch?v=JZHJ2679XMc&t=...,NaN,NaN,NaN,NaN
1656,2123,shit,spread on the door five and six take your pick...,spread on the door five and six take your pick...,3246,https://www.youtube.com/watch?v=JZHJ2679XMc&t=...,NaN,NaN,NaN,NaN
1657,608,dope,boss i dont know who that is guys do you guys ...,boss yeah i dont know that is you guys you guy...,4331,https://www.youtube.com/watch?v=PKQ6B7r2TrI&t=...,NaN,NaN,NaN,NaN


In [ ]:
df[df['Slur'] == "anus"]

,Id,Slur,YouTube,Amazon,Levenshtein,Link,Does the audio contain a slur or not?,Original Transcription,"Race of speaker (Caucasian, Asian, South Asian, Black, and Australoid/ Animation)",Actual Word
485,0,anus,has in hurry nice now we are ready to put our ...,has in her anus now we are ready to put our do...,14,https://www.youtube.com/watch?v=t9rMyBQlvZc&t=...,NaN,NaN,NaN,NaN


In [ ]:
list(df[df['Slur'] == "anus"]['YouTube']), list(df[df['Slur'] == "anus"]['Amazon'])

(['has in hurry nice now we are ready to put our doll together'],
 ['has in her anus now we are ready to put our doll together'])

In [ ]:
with open("/content/slurs.txt", "w") as file:
    for s in list(set(df['Slur'])):
        file.write("%s\n"%s)

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-large-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
#constructing a vocabulary
vocabulary=[]
for i in range(len(tokenizer)):
  vocabulary.append(tokenizer.decode([i]))

In [ ]:
len(vocabulary)

28996

In [ ]:
#constructs a set of unique words from the given string
def unique_words(x):
  temp=[]
  for i in x:
    if i.lower() not in temp:
      temp.append(i.lower())
  return temp

#proposes a set of candidate words based on a condition (here, the Levenshtein distance is <=1) and the most likely words as proposed by the MLM
def candidate_set(word_, mask_logits):
  candidate_scores={}
  word_limit, prim_cand=int(len(word_)/2), []

  for item in vocabulary:
    lev=textdistance.levenshtein(word_, item)
    if(lev<=word_limit):
      id=vocabulary.index(item)
      candidate_scores[item]=mask_logits[0, id].item()
    if(lev<=1):
      prim_cand.append(item)

  return candidate_scores, prim_cand

#returns a list of the N most likely candidates, where N is specified by the user
def top_cand(dict_, top_tokens):
    top_candidates=list(dict_.keys())

    if(len(top_candidates)<top_tokens):
      top_tokens=len(top_candidates)

    return [top_candidates[i] for i in range(top_tokens)]
    #return [tokenizer.decode(top_candidates[i]) for i in range(top_tokens)]


def remove_from_dict(diction, video_id, prim_cand):
  to_be_removed=[]
  matching_list=dict_matched_unique_words[video_id]
  #print(matching_list)
  for i in diction.keys():
    if((i not in matching_list) and (i not in prim_cand)):
      to_be_removed.append(i)

  for i in to_be_removed:
    diction.pop(i)

  return diction

#returns the probability
def predict_prob(word_, mask_logits):
      predict_token=tokenizer.encode(word_)
      predict_token=predict_token[1]

      return mask_logits[0, predict_token].item()

#returning the list of top N candidates
def find_top(diction):
  keys_=list(diction.keys())
  values_=list(diction.values())
  w1,v1=[], []

  try:
    for x in range(topk):
      w1.append(keys_[x])
      v1.append(values_[x])
  except:
    return (w1, v1)

  return (w1, v1)

#sorting a dictionary by descending order based on its value
def sort_di(dict_name):
  keylist={k: v for k, v in sorted(dict_name.items(), key=lambda item: item[1], reverse=True)}
  return keylist

In [ ]:
#Getting all the unique words for each transcript and saving them into a dictionary
matched=pd.read_csv("CSV/Matched Transcripts.csv")
act=df['Actual Word'].to_list()
act=[i for i in act if i==i]
act=unique_words(act)
print(act)

['maya', 'copter', 'crab', 'craft', 'grumpy', 'adult', 'race is', 'version', 'ask', 'dinner', 'done', 'trunk', 'duck', 'heroine', 'thing', 'pink', 'rake', 'shirt', 'sit', 'ah', 'beach', 'benches', 'cut', 'trash', 'dent', 'full', 'fun', 'bugs', 'pose', 'rain', 'shed', "she's", 'fish', 'oops', 'vlogger', 'dough', 'draw', 'draft', 'fox', 'look', 'looking', 'kells', 'venus', 'pew', 'sacks', 'sug', 'sleigh', 's', 'adds', 'orbit shh', 'bin', 'bet', 'pitch', 'bootsie', 'birdy', 'cockadoodledo', 'toyitis', 'digly', 'drum', 'job', 'strong', 'box', 'popped', 'pumpkins', 'getting', 'little', 'lick of', 'neck-to-neck', 'pets', 'rainbow corn', 'rubbed', 'set', 'said', 'kinder egg', 'as', 'as is', 'djibouti', 'playdoh', 'door', 'draculaura', 'round cut', 'bag', 'pluck me', 'kindi', 'barbie doll oh', 'moana', 'nuggets', 'to test', 'for more', 'peel', 'rainbow', 'race', 'cents', 'socks', 'ship', 'firm', 'whoosh', 'last', 'us', 'a blue', "grandma's", 'd meal', "digly's", 'bucket', 'buckle', 'okay', 'th

In [ ]:
len(act)

178

In [ ]:
df

,Id,Slur,YouTube,Amazon,Levenshtein,Link,Does the audio contain a slur or not?,Original Transcription,"Race of speaker (Caucasian, Asian, South Asian, Black, and Australoid/ Animation)",Actual Word
0,53,ass,ready for a rescue shes going to take to the s...,ready for a rescue shes going to take to the s...,0,https://www.youtube.com/watch?v=0Z9rIS4xNZ0&t=...,No,ready for a rescue shes going to take to the s...,Caucasian Woman,maya
1,218,bitch,his life hey bitch mean,his life hey bitch mean,0,https://www.youtube.com/watch?v=9R4lLcm5p7I&t=...,NaN,NaN,NaN,NaN
2,312,cocktail,also exceptionally strong for their size they ...,also exceptionally strong for their size they ...,0,https://www.youtube.com/watch?v=Fw1_1_yKZ78&t=...,No,also exceptionally strong for their size they ...,NaN,copter
3,326,crap,the red crap is standing on,the red crap is standing on,0,https://www.youtube.com/watch?v=pmQTB3ScUo8&t=...,No,The red crab is standing on,NaN,crab
4,360,crap,stretchy and sticky and now we have a crap and...,stretchy and sticky and now we have a crap and...,0,https://www.youtube.com/watch?v=jrZwy-ybyfA&t=...,No,stretchy and sticky and now we have a crab and...,NaN,crab
...,...,...,...,...,...,...,...,...,...,...
1654,2110,shit,just for fun climb and ten nine ten dont you w...,just for fun fine and 10 9 and 10 dont you wan...,2643,https://www.youtube.com/watch?v=zwibXy4CCgo&t=...,NaN,NaN,NaN,NaN
1655,1862,rape,spread on the door five and six take your pick...,spread on the door five and six take your pick...,3246,https://www.youtube.com/watch?v=JZHJ2679XMc&t=...,NaN,NaN,NaN,NaN
1656,2123,shit,spread on the door five and six take your pick...,spread on the door five and six take your pick...,3246,https://www.youtube.com/watch?v=JZHJ2679XMc&t=...,NaN,NaN,NaN,NaN
1657,608,dope,boss i dont know who that is guys do you guys ...,boss yeah i dont know that is you guys you guy...,4331,https://www.youtube.com/watch?v=PKQ6B7r2TrI&t=...,NaN,NaN,NaN,NaN


In [ ]:
matched.head(3)

,Unnamed: 0,Video Id,Amazon,YouTube,Matched Transcript,Match Score
0,0,eSFxZzyH5nM,Oh hey thank you Thank you Welcome to open sta...,HOOTS: Oh! AUDIENCE: [APPLAUSE] HOOTS: Hey you...,hey thank welcome stage at everyone can step u...,0.518750
1,1,M4YSHQv0pqM,I scream too much sugar What That's not just o...,[Music] [Music] their days so we feed him some...,you were play doesnt spit comes here put face ...,0.289474
2,2,1-3xYUwNxeQ,I don't remember how he looks like What does r...,I don't remember how he looked like what does ...,remember how he does youre original welcome hi...,0.495614


In [ ]:
#creating a list of words in the Amazon and YouTube dictiona
dict_matched={}
dict_matched_unique_words={}
for i in range(len(matched)):
  #dict_matched[matched['Video Id'][i]]=(matched['Amazon'][i], matched['YouTube'][i])
  words_in_transcript=(str(matched['Amazon'][i]) + str(matched['YouTube'][i])).lower().split()
  words_in_transcript=unique_words(words_in_transcript)
  for item in act:
    if(item+'s') in words_in_transcript and item not in words_in_transcript:
      words_in_transcript.append(item)
  dict_matched_unique_words[matched['Video Id'][i]]=words_in_transcript

In [ ]:
#dict_matched_unique_words['18tQBHqsw-A']

In [ ]:
#Masking the words and predicting the r
def print_results(df, type_, top_k_words):
    #the masked token will vary with every model
    repl_word =" [MASK] "
    count=0
    n_count=0
    cand_count=0

    for i in range(len(df)):
      try:

          actual_word=df['Actual Word'][i]
          if((df['Does the audio contain a slur or not?'][i]!='No') or (actual_word=='[Noises]')):
            continue
          am=str(df[type_][i])
          word=str(df['Slur'][i])
          vid_id=str(df['Link'][i])[32:43]

          #replaces the word with the masked token
          res = am.replace(word, repl_word, 1)
          input = tokenizer.encode(res, return_tensors="pt")
          mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
          #predicts the masked token
          token_logits = model(input).logits
          mask_token_logits = token_logits[0, mask_token_index, :]

          #print("Amazon:", res)
          #print("Original: ", predict_prob(actual_word, mask_token_logits), actual_word)
          #print("Slur: ", predict_prob(word, mask_token_logits), word)

          count+=1
          if(predict_prob(word, mask_token_logits)<=predict_prob(df['Actual Word'][i], mask_token_logits)):
            n_count+=1

          dict_, primary_candidates= candidate_set(word, mask_token_logits)
          dict_= remove_from_dict(dict_, vid_id, primary_candidates)
          dict_=sort_di(dict_)

          candidate_list=top_cand(dict_, top_k_words)
          #print(candidate_list)

          scores.append(find_top(dict_))

          ground_truth.append(actual_word)
          video_ids_list.append(vid_id)
          if(actual_word in candidate_list):
            print("Present in top candidate list!")
            present.append('Present')
            cand_count+=1
          else:
            present.append('Absent')

          #print('\n\n\n')
      except Exception as e:
       print(e)

    print("Total: ", count)
    print("Suggestions that had a greater probability than the slur word: ", n_count)
    print("Ratio: ", n_count/count)
    print("Candidate Test Success Count: ", cand_count)

In [ ]:
#For instances
scores=[]
ground_truth=[]
present=[]
video_ids_list=[]
#This parameters signifies the value of p. topk=1 signifies P-1, topk=3 signifies P-3 and so forth
topk=1
print_results(df, "YouTube", topk)

Present in top candidate list!
Present in top candidate list!
Present in top candidate list!
Present in top candidate list!
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
Present in top candidate list!
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
index 0 is out of bounds for dimension 0 with size 0
Present in top candidate list!
Present in top cand

In [ ]:
#saving the results into a CSV file
list_entries=[]
for i in range(len(scores)):
  list_entries.append((video_ids_list[i], ground_truth[i], scores[i][0], scores[i][1], present[i]))

df_to_save = pd.DataFrame(list_entries, columns =['Video Id', 'Ground Truth','Words','Scores', 'Present'])
df_to_save.to_csv("CSV/Results/P1_Amazon_Normalized_AddPluralForms.csv")

In [ ]:
df_to_save

,Video Id,Ground Truth,Words,Scores,Present
0,0Z9rIS4xNZ0,maya,[ass],[2.9170236587524414],Absent
1,Fw1_1_yKZ78,copter,[local],[0.4974931478500366],Absent
2,pmQTB3ScUo8,crab,[car],[6.567646026611328],Absent
3,jrZwy-ybyfA,crab,[car],[7.778393268585205],Absent
4,irlvY_fyNtQ,craft,[craft],[3.3310887813568115],Present
5,WnijkuQ83TA,craft,[cup],[2.554880142211914],Absent
6,u5F6eTSvQjw,craft,[craft],[2.0116078853607178],Present
7,WTdi0S4t-XM,craft,[can],[1.422349214553833],Absent
8,SnzwfLHGjSg,craft,[trap],[2.293825149536133],Absent
9,eiIBr_MueIo,craft,[can],[5.627756118774414],Absent


In [ ]:
len(df_to_save[df_to_save['Present'] == "Present"]) / len(df_to_save)

0.18181818181818182